# 🏠House 💵prices predictions with Pytorch Lightning⚡

In [1]:
from pytorch_lightning import LightningDataModule, Trainer, LightningModule
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
import numpy as np
import warnings
from torch import nn
import torch.nn.functional as F
from torchmetrics import Accuracy, Precision, Recall, F1Score
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
import itertools


## Function: `classify_house`

This function classifies a house based on its year of construction, remodeling year, style, and building type. It takes a single argument `row`, which is expected to be a dictionary or Pandas Series containing house attributes.

### Parameters:

- `row`: A dictionary or Pandas Series with keys representing column names and values representing data for a single house.

### Process and Logic:

1. **Age Classification:**
   - The house is classified as `'Modern'` if it was built or remodeled in the year 2000 or later.
   - Otherwise, it is classified as `'Old'`.

2. **Style Classification:**
   - Houses with a style of `'2Story'` or `'2.5Fin'` are classified as `'Multi Level'` to indicate multiple living levels.
   - Houses styled as `'1Story'` or `'1.5Fin'` are classified as `'Single Level'`.
   - All other styles are classified as `'Other'`.

3. **Building Type Classification:**
   - Houses identified as `'1Fam'` are classified as `'Single Family'` homes.
   - Building types of `'Duplex'` or `'2fmCon'` are classified as `'Multi Family'`.
   - All remaining types are grouped under `'Townhouse'`.

### Return Value:

- The function returns a string that combines all three classifications, formatted as `"age_category style_category type_category"`.

### Example Usage:

```python
# Example house data
house_data = {
    'YearBuilt': 2005,
    'YearRemodAdd': 2010,
    'HouseStyle': '2Story',
    'BldgType': '1Fam'
}

# Classify the house
house_description = classify_house(house_data)
print(house_description)


In [2]:
def classify_house(row):
    if row['YearBuilt'] >= 2000 or row['YearRemodAdd'] >= 2000:
        age_category = 'Modern'
    else:
        age_category = 'Old'

    if row['HouseStyle'] in ['2Story', '2.5Fin']:
        style_category = 'Multi Level'
    elif row['HouseStyle'] in ['1Story', '1.5Fin']:
        style_category = 'Single Level'
    else:
        style_category = 'Other'

    if row['BldgType'] == '1Fam':
        type_category = 'Single Family'
    elif row['BldgType'] in ['Duplex', '2fmCon']:
        type_category = 'Multi Family'
    else:
        type_category = 'Townhouse'

    return f"{age_category} {style_category} {type_category}"

The given script utilizes Python's `pandas` library and `warnings` module to perform a series of data manipulation tasks. Below is a step-by-step explanation of each operation:

1. **Suppressing Warnings:**
   The code begins by suppressing all warnings that might be generated by Python. This is typically done to clean up the output, making it free from any warning messages that might not be crucial to address immediately.

2. **Loading Data:**
   The script reads the same CSV file `train.csv` twice, each time loading its contents into two separate DataFrames, `pdDf1` and `pdDf2`. This approach can be useful for testing or when simulating a larger dataset by concatenating the same data.

3. **Concatenating DataFrames:**
   It then concatenates these two DataFrames along the vertical axis (rows). This effectively doubles the dataset by stacking `pdDf2` directly below `pdDf1`. The resulting DataFrame, `pdDf`, now contains twice the number of rows originally present in `train.csv`.

4. **Displaying the First Few Rows:**
   Finally, the script displays the first five rows of the newly concatenated DataFrame using the `.head()` method. This is a common practice to quickly verify the top rows of the DataFrame, ensuring the concatenation was performed correctly and to visually confirm the data structure.


In [3]:
warnings.filterwarnings('ignore')

pdDf1 = pd.read_csv('train.csv')
pdDf2 = pd.read_csv('train.csv')
pdDf = pd.concat([pdDf1, pdDf2])
pdDf.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Python Code Explanation: Selecting Specific Columns in DataFrame

This segment of Python code focuses on working with a pandas DataFrame to select a subset of its columns based on specific interests or data analysis requirements. Here's what each part of the script achieves:

1. **Defining Column Keys:**
   The code defines a list named `keys` that contains the names of the columns to be retained or focused on in the DataFrame. These column names include various attributes of houses such as 'Neighborhood', 'HouseStyle', 'YearBuilt', 'YearRemodAdd', and several others related to the structure and sale details of the houses.

2. **Selecting Columns from DataFrame:**
   Instead of dropping columns (as hinted by the commented-out code), the script selects only those columns listed in `keys` from the DataFrame `pdDf`. This operation simplifies the DataFrame by reducing its dimensionality to only include columns that are likely relevant for further analysis or modeling:
   ```python
   pdDf = pdDf[keys]


In [4]:
keys = ["Neighborhood", "HouseStyle" , "YearBuilt" , "YearRemodAdd" , "Exterior1st" ,
        "Exterior2nd" , "MasVnrType" , "Foundation" , "BsmtFinType1" , "1stFlrSF", "Functional" ,
        "GarageType", "BldgType", "SalePrice"]
# pdDf = pdDf.drop(columns=keys)
pdDf = pdDf[keys]

In [5]:
pdDf['House Category'] = pdDf.apply(classify_house, axis=1)

### Handling Missing Values in DataFrame

1. **Iterating Through Columns:**
   - The loop `for i in pdDf.columns:` allows the code to process each column individually.

2. **Condition Based on Data Type:**
   - If the column data type is `int64` or `float64` (numeric types), the missing values are filled with the mean of the column:
     ```python
     pdDf[i].fillna(pdDf[i].mean(), inplace=True)
     ```
     This method computes the mean of the column and uses it to replace all `NaN` values, which is suitable for numerical data as it maintains the statistical properties of the column.

   - If the column data type is `object` (typically categorical or textual data), the missing values are filled with the mode of the column:
     ```python
     pdDf[i].fillna(pdDf[i].mode()[0], inplace=True)
     ```
     The mode (the most frequently occurring value in the column) is used here, which is a common practice for categorical data to ensure consistency with the most common category.

3. **In-Place Modification:**
   - The `inplace=True` argument modifies the DataFrame `pdDf` directly, saving memory and keeping changes within the original DataFrame without needing to create a copy.


In [6]:
for i in pdDf.columns:
    if pdDf[i].dtypes == "int64" or pdDf[i].dtypes == "float64" :
        pdDf[i].fillna(pdDf[i].mean(), inplace=True)
    elif pdDf[i].dtypes == "object":
        pdDf[i].fillna(pdDf[i].mode()[0], inplace=True)

### Encoding Categorical Columns in a DataFrame

This code snippet demonstrates the process of encoding categorical columns in a pandas DataFrame using `LabelEncoder` from the `sklearn` library:

1. **Initialization of LabelEncoder:**
   - `le = LabelEncoder()`: This initializes a `LabelEncoder` object, which is used to convert categorical labels into numerical format. This transformation is necessary for many machine learning models that require numerical input.

2. **Identifying Categorical Columns:**
   - `categorical_cols = pdDf.select_dtypes(include=['object']).columns`: This line identifies all columns in the DataFrame `pdDf` that have data type `object` (commonly used for textual or categorical data) and stores their names in `categorical_cols`.

3. **Applying LabelEncoder:**
   - The loop `for col in categorical_cols:` iterates through each categorical column.
   - `pdDf[col] = le.fit_transform(pdDf[col])`: For each column, `LabelEncoder` is applied to transform the textual data into a numerical format. The `fit_transform` method first fits the label encoder to the data, then transforms it, replacing the original textual data with corresponding numerical codes directly in the DataFrame.


In [7]:
# Initialize LabelEncoder
le = LabelEncoder()

# List of columns to encode
categorical_cols = pdDf.select_dtypes(include=['object']).columns

# Apply LabelEncoder to each categorical column
for col in categorical_cols:
    pdDf[col] = le.fit_transform(pdDf[col])

### Standardizing the "SalePrice" Column in a DataFrame

In [9]:
price_mean = np.nanmean(pdDf["SalePrice"])
price_std = np.nanstd(pdDf["SalePrice"])
print(f"mean: {price_mean} ; STD: {price_std}")

pdDf["SalePrice"] = (pdDf["SalePrice"] - price_mean) / price_std

mean: 180921.19589041095 ; STD: 79415.29188606751


In [10]:
pdDf["House Category"].unique()

array([ 1, 15,  9,  6, 11,  3, 14, 12, 16,  7,  0,  2, 13, 10,  8,  4,  5])

### Class: HouseDataModule

`HouseDataModule` is a custom class designed to handle data operations for a housing dataset in a PyTorch Lightning framework. It encapsulates functionality for data preprocessing, splitting, and loading, making it suitable for training, validation, and testing phases of machine learning models.

#### Constructor: `__init__`
- Initializes the data module with a DataFrame, batch sizes for loading, proportions for test and validation splits, and a random seed for reproducibility.
- A `LabelEncoder` is instantiated for use in categorical data encoding.

#### Method: `prepare_data`
- Scales numeric features using `StandardScaler`, excluding the target column 'SalePrice'. This normalization standardizes the feature set to have zero mean and unit variance, a common requirement for many ML algorithms.

#### Method: `setup`
- Splits the DataFrame into training, validation, and test sets using `train_test_split`. This division is critical for evaluating model performance effectively.
- Prepares tensor datasets for each split by extracting features and labels, and converting them into PyTorch tensors, facilitating their use in neural network training.

#### Data Loaders
- `train_dataloader`, `val_dataloader`, `test_dataloader`: These methods return PyTorch `DataLoader` objects for the respective datasets. These loaders provide batched, optionally shuffled data to the model during training or evaluation.
- `predict_dataloader`: Returns a DataLoader for the test dataset, which can be used for making predictions post-training.

### Usage
- The `HouseDataModule` class is used within a PyTorch Lightning training pipeline, where it provides preprocessed and loaded data to the model during different phases of training and evaluation.

### Purpose
- This class abstracts away the complexities of data handling, allowing the main focus to be on model architecture and training logic. It ensures data is correctly preprocessed and split, providing a robust foundation for building reliable machine learning models.


In [11]:
class HouseDataModule(LightningDataModule):
    def __init__(self, dataframe, batch_size=32, test_size=0.2, val_size=0.1, random_state=42):
        super().__init__()
        self.df = dataframe
        self.batch_size = batch_size
        self.test_size = test_size
        self.val_size = val_size
        self.random_state = random_state
        self.label_encoder = LabelEncoder()

    def prepare_data(self):

        # Scale the numeric features
        self.scaler = StandardScaler()
        numeric_cols = self.df.select_dtypes(include=['int64', 'float64']).columns.drop(['SalePrice'])
        self.df[numeric_cols] = self.scaler.fit_transform(self.df[numeric_cols])

    def setup(self, stage=None):
        # Split the data into training+validation and test sets
        train_val_df, test_df = train_test_split(self.df, test_size=self.test_size, random_state=self.random_state)

        # Split training+validation into actual training and validation sets
        train_df, val_df = train_test_split(train_val_df, test_size=self.val_size / (1 - self.test_size), random_state=self.random_state)
        
        # Prepare training data
        X_train = train_df.drop(['SalePrice', 'House Category'], axis=1).values
        y_reg_train = train_df['SalePrice'].values
        y_cls_train = train_df['House Category'].values
        self.train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float),
                                           torch.tensor(y_reg_train, dtype=torch.float),
                                           torch.tensor(y_cls_train, dtype=torch.long))

        # Prepare validation data
        X_val = val_df.drop(['SalePrice', 'House Category'], axis=1).values
        y_reg_val = val_df['SalePrice'].values
        y_cls_val = val_df['House Category'].values
        self.val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float),
                                         torch.tensor(y_reg_val, dtype=torch.float),
                                         torch.tensor(y_cls_val, dtype=torch.long))

        # Prepare test data
        X_test = test_df.drop(['SalePrice', 'House Category'], axis=1).values
        y_reg_test = test_df['SalePrice'].values
        y_cls_test = test_df['House Category'].values
        self.test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float),
                                          torch.tensor(y_reg_test, dtype=torch.float),
                                          torch.tensor(y_cls_test, dtype=torch.long))

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)


### Class: HouseNet

`HouseNet` is a neural network model implemented using PyTorch Lightning, designed to handle both regression and classification tasks simultaneously. This model architecture is particularly useful for datasets where you need to predict both a numerical value and a categorical label from the same set of input features.

#### Constructor: `__init__`
- **Parameters:**
  - `input_size`: The number of input features.
  - `num_classes`: The number of categories for the classification task.
  - `dropout_rate`: Dropout rate to prevent overfitting.
  - `lr`: Learning rate for the optimizer.
- **Operations:**
  - Initializes shared layers that are common to both tasks, which helps in learning representations that are useful across both regression and classification.
  - Initializes separate heads for regression and classification tasks. The regression head predicts a continuous variable, and the classification head predicts categorical labels.
  - Initializes metrics for tracking the performance of the classification task during training and testing, such as accuracy, precision, recall, and F1 score.

#### Forward Method: `forward`
- **Functionality:**
  - Takes input features and passes them through the shared layers.
  - Outputs from the shared layers are fed into both the regression and classification heads.
  - Returns the predictions from both heads.

#### Training Step: `training_step`
- Computes losses for both regression and classification tasks and combines them into a single loss.
- Logs training loss and accuracy and appends them to the internal history for monitoring.

#### Validation and Test Steps: `validation_step`, `test_step`
- Similar to the training step, these methods compute and log losses and metrics for validation and testing phases.
- They also log precision, recall, and F1 scores for the classification task.

#### Metric History:
- Maintains a history of various metrics for training, validation, and testing phases to monitor model performance over epochs.

#### Optimizer Configuration: `configure_optimizers`
- Configures an Adam optimizer with a learning rate scheduler to adjust the learning rate during training, optimizing the training process.

### Summary

The `HouseNet` model is structured to effectively address both regression and classification challenges within a single framework. By leveraging shared layers, the model efficiently learns representations that are useful for both predicting a continuous variable and classifying input data into categories. This dual-task approach is beneficial in scenarios where the outputs are interdependent or when training on datasets that support multiple objectives.


In [12]:


class HouseNet(LightningModule):
    def __init__(self, input_size, num_classes, dropout_rate=0.5, lr = 0.01):
        super().__init__()
        self.save_hyperparameters()

        # Shared layers
        self.shared_layers = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate)
        )
        self.lr = lr
        # Regression head
        self.reg_head = nn.Linear(64, 1)
        
        # Classification head
        self.cls_head = nn.Linear(64, num_classes)

        self.metrics_history = {'train_loss': [], 'val_loss': [], 'train_accuracy': [], 
                                'val_accuracy': [], 'val_precision': [], 'val_recall': [], 'val_f1_score': [],
                               'test_loss': [], 'test_accuracy': [], 'test_precision': []}

        # Metrics
        self.accuracy = Accuracy(task='multiclass', num_classes=num_classes, average='macro')
        self.precision = Precision(task='multiclass', num_classes=num_classes, average='macro')
        self.recall = Recall(task='multiclass', num_classes=num_classes, average='macro')
        self.f1_score = F1Score(task='multiclass', num_classes=num_classes, average='macro')

    def forward(self, x):
        shared_out = self.shared_layers(x)
        reg_pred = self.reg_head(shared_out)
        cls_pred = self.cls_head(shared_out)
        return reg_pred, cls_pred

    def training_step(self, batch, batch_idx):
        x, y_reg, y_cls = batch
        reg_pred, cls_pred = self(x)
        reg_loss = F.mse_loss(reg_pred.squeeze(), y_reg)
        cls_loss = F.cross_entropy(cls_pred, y_cls)
        loss = reg_loss + cls_loss
        accuracy = self.accuracy(cls_pred, y_cls)
        # Log and store
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('train_accuracy', accuracy, on_step=False, on_epoch=True, prog_bar=True)
        self.metrics_history['train_loss'].append(loss.detach())
        self.metrics_history['train_accuracy'].append(accuracy.detach())
        return loss

    def test_step(self, batch, batch_idx):
        x, y_reg, y_cls = batch
        reg_pred, cls_pred = self(x)
        reg_loss = F.mse_loss(reg_pred.squeeze(), y_reg)
        cls_loss = F.cross_entropy(cls_pred, y_cls)
        loss = reg_loss + cls_loss

        self.metrics_history['test_loss'].append(loss.detach())
        self.metrics_history['test_accuracy'].append(self.accuracy(cls_pred, y_cls).detach())
        
        self.log('test_loss', loss , on_epoch=True)
        self.log('test_accuracy', self.accuracy(cls_pred, y_cls), on_epoch=True)
        self.log('test_precision', self.precision(cls_pred, y_cls), on_epoch=True)
        self.log('test_recall', self.recall(cls_pred, y_cls), on_epoch=True)
        self.log('test_f1_score', self.f1_score(cls_pred, y_cls), on_epoch=True)
        return {"test_loss": loss, "test_accuracy": self.accuracy(cls_pred, y_cls)}

    def validation_step(self, batch, batch_idx):
        x, y_reg, y_cls = batch
        reg_pred, cls_pred = self(x)
        reg_loss = F.mse_loss(reg_pred.squeeze(), y_reg)
        cls_loss = F.cross_entropy(cls_pred, y_cls)
        # Log metrics
        self.log('val_loss', reg_loss + cls_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_accuracy', self.accuracy(cls_pred, y_cls), on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_precision', self.precision(cls_pred, y_cls), on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_recall', self.recall(cls_pred, y_cls), on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_f1_score', self.f1_score(cls_pred, y_cls), on_step=False, on_epoch=True, prog_bar=True)
        return {"val_loss": reg_loss + cls_loss, "val_acc": self.accuracy(cls_pred, y_cls)}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), self.lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        return [optimizer], [scheduler]


### Hyperparameter Tuning with PyTorch Lightning

This script outlines a comprehensive approach for conducting hyperparameter tuning on a machine learning model built with PyTorch Lightning. It leverages various features of the framework to automate and streamline the experimentation process.

#### Data Module Initialization
- **`HouseDataModule`** is instantiated with the DataFrame `pdDf` and a specified batch size. This module handles all data loading and preprocessing:
  - `prepare_data()` method scales numeric features.
  - `setup()` method splits the data into training, validation, and test sets.

#### Model Configuration
- The input size and number of classes are determined based on the DataFrame structure:
  - `input_size` is set based on the number of features, excluding target columns like 'SalePrice' and 'House Category'.
  - `num_classes` is determined by the number of unique categories in the 'House Category' column.

#### Hyperparameter Setup
- A dictionary `hyperparams` is defined, listing potential values for learning rates, dropout rates, and batch sizes.
- Combinations of these hyperparameters are generated to explore different configurations.

#### Training Loop
- For each combination of hyperparameters:
  - A unique `TensorBoardLogger` is created to log training results for each configuration, facilitating performance comparison.
  - A new `HouseDataModule` is instantiated with the current batch size from the hyperparameter set.
  - `HouseNet`, the model, is initialized with the current set of hyperparameters.
  - A `Trainer` is set up with a checkpoint callback to save the best model and a learning rate monitor.
  - The model is trained using the `fit` method of the `Trainer`.
  - After training, the model undergoes validation and testing. Results are logged automatically and can be reviewed in TensorBoard.

#### Output
- The script prints a summary for each training session indicating the hyperparameter configuration used, and it executes validation and testing to assess model performance under each configuration.

### Purpose
- This script is ideal for scenarios where finding the optimal model configuration is crucial. It systematically explores a range of model settings to identify the best performing parameters in terms of learning rate, dropout rate, and batch size. The extensive logging with TensorBoard allows for easy comparison of results across different runs.

### Execution
- By iterating over a predefined grid of hyperparameters and logging results for each combination, this method provides a thorough means of evaluating how different configurations affect model performance, making it invaluable for fine-tuning model parameters to achieve the best results.


In [13]:

data_module = HouseDataModule(pdDf, batch_size=32)
data_module.prepare_data()
data_module.setup()

# Assuming df is already available and appropriate transformations are applied
input_size = len(pdDf.drop(['SalePrice', 'House Category'], axis=1).columns)
num_classes = len(pdDf["House Category"].unique())
# Define hyperparameter grid (as dictionary)
hyperparams = {
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout_rate': [0.3, 0.5],
    'batch_size': [32, 64]
}

# Generate all combinations of hyperparameters
keys, values = zip(*hyperparams.items())
permutations = [dict(zip(keys, v)) for v in itertools.product(*values)]

for config in permutations:
    # Create a unique logger for each hyperparameter configuration
    logger = TensorBoardLogger('tb_logs', name=f"model_lr={config['learning_rate']}_dropout={config['dropout_rate']}_batch={config['batch_size']}")

    # Update data module and model with current config
    model = HouseNet(input_size=input_size, num_classes=num_classes, dropout_rate=config['dropout_rate'], lr=config['learning_rate'])
    data_module = HouseDataModule(pdDf, batch_size=config['batch_size'])

    # Setup trainer with current logger and checkpoint callback
    checkpoint_callback = ModelCheckpoint(dirpath=logger.log_dir, monitor="val_loss", mode="min")
    lr_monitor = LearningRateMonitor(logging_interval='epoch')
    trainer = Trainer(
        logger=logger,
        callbacks=[checkpoint_callback, lr_monitor],
        max_epochs=50,
        deterministic=True
    )

    # Train the model with the current set of hyperparameters
    trainer.fit(model, datamodule=data_module)

    # Optionally, you can run validation or testing here as well
    print(f"model_lr={config['learning_rate']}_dropout={config['dropout_rate']}_batch={config['batch_size']}")
    trainer.validate(model, datamodule=data_module)
    trainer.test(model, datamodule=data_module)
    


Missing logger folder: tb_logs\model_lr=0.001_dropout=0.3_batch=32
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type                | Params
------------------------------------------------------
0 | shared_layers | Sequential          | 10.0 K
1 | reg_head      | Linear              | 65    
2 | cls_head      | Linear              | 1.1 K 
3 | accuracy      | MulticlassAccuracy  | 0     
4 | precision     | MulticlassPrecision | 0     
5 | recall        | MulticlassRecall    | 0     
6 | f1_score      | MulticlassF1Score   | 0     
------------------------------------------------------
11.2 K    Trainable params
0         Non-trainable params
11.2 K    Total params
0.045     Total estimated model params size (MB)


Sanity Checking: |                                                                               | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

model_lr=0.1_dropout=0.5_batch=64


Validation: |                                                                                    | 0/? [00:00<…

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_accuracy        │    0.09675469249486923    │
│       val_f1_score        │   0.049513086676597595    │
│         val_loss          │     3.117926597595215     │
│       val_precision       │    0.03364440053701401    │
│        val_recall         │    0.09675469249486923    │
└───────────────────────────┴───────────────────────────┘

Testing: |                                                                                       | 0/? [00:00<…

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.09576193243265152    │
│       test_f1_score       │    0.04470125213265419    │
│         test_loss         │    3.0932531356811523     │
│      test_precision       │   0.029454587027430534    │
│        test_recall        │    0.09576193243265152    │
└───────────────────────────┴───────────────────────────┘

In [19]:
len(pdDf['House Category'].unique())

17